# La libreria DEAP

DEAP (abreviatura de Algoritmos Evolutivos Distribuidos en Python) es un marco de trabajo de Python que facilita el desarrollo rápido de soluciones mediante algoritmos genéticos, así como otras técnicas de computación evolutiva.

DEAP ofrece diversas estructuras de datos y herramientas que resultan esenciales al implementar una amplia gama de soluciones basadas en algoritmos genéticos.

In [ ]:
 pip install deap

In [5]:
import numpy as np

## Usando el modulo creador

La primera herramienta potente que ofrece el framework DEAP es el módulo creador. Este módulo permite ampliar las clases existentes, complementándolas con nuevos atributos.  Por ejemplo si tenemos una clase llamada empleado, podemos ampliar con la herramienta creadora, la clase Empleado creando una clase llamada Desarrollador de la siguiente manera:

In [15]:
##Defino primero mi clase
class Empleado:
    def __init__(self, nombre):
        self.nombre = nombre

#creo la nueva clase Desarrollador que hereda de Empleado y le agrego dos atributos nuevos
from deap import creator
creator.create("Desarrollador", Empleado, position="Desarrollador", programmingLanguages=set)



## Creando la función Fitness

Al usar DEAP, los valores de aptitud física se encapsulan en una clase de aptitud física. DEAP permite combinar la aptitud física en varios componentes (también llamados objetivos), cada uno con su propio peso. La combinación de estos pesos define el comportamiento o la estrategia de aptitud física para el problema dado.


Para definir esta estrategia, DEAP incluye la clase abstracta base.Fitness, que contiene una tupla de pesos. Es necesario asignar valores a esta tupla para definir la estrategia y que la clase sea utilizable. Esto se logra extendiendo la clase base Fitness mediante el creador, de forma similar a como hicimos con la clase Developer anterior:

In [25]:
### Creando la función Fitness
from deap import base

#Esta función crea una nueva clase llamada FitnessMax que hereda de base.Fitness. 
# El atributo weights se establece en (1.0,), lo que indica que se trata de un problema de maximización con un solo objetivo observa el peso que se da
creator.create("FitnessMax", base.Fitness, weights=(1.0,))  

# De manera similar, podemos crear una clase para problemas de minimización, observa el peso negativo que se da
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))


c:\Users\javie\AppData\Local\Programs\Python\Python313\Lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


Esto generará una clase creator.FitnessMax que extiende la clase base.Fitness, con el atributo de clase weights inicializado a un valor de (1.0,)

También podemos definir una clase con una estrategia para optimizar más de un objetivo, y con distintos grados de importancia. Esto generará una clase creator.FitnessCompound que utilizará tres componentes de fitness diferentes. El primero tendrá un peso de 1.0, el segundo de 0.2 y el tercero de -0.5. Esto tenderá a maximizar el primer y el segundo componente (u objetivos) y a minimizar el tercero. En términos de importancia, el primer componente tiene la mayor importancia, seguido del tercero y el segundo.

In [27]:
from deap import base

# Esto generará una clase creator.FitnessCompound que utilizará tres componentes de fitness diferentes.
creator.create("FitnessCompound", base.Fitness, weights=(1.0, 0.2, -0.5))


c:\Users\javie\AppData\Local\Programs\Python\Python313\Lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'FitnessCompound' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


## Almacenamiento de los valores fitness

Mientras que la tupla de "weights" define la estrategia de aptitud, se utiliza una tupla coincidente, denominada "values", para contener los valores de aptitud reales dentro de la clase base.Fitness. Estos valores se obtienen de una función definida por separado, normalmente denominada "evaluate()", como se describirá más adelante. Al igual que la tupla de "weights", la tupla de "values" contiene un valor para cada componente de aptitud (objetivo).


Una tercera tupla, "wvalues", contiene los valores ponderados obtenidos al multiplicar cada componente de la tupla values ​​con su componente correspondiente de la tupla weights. Al establecer los valores de aptitud de una instancia, estos se calculan y se insertan en wvalues. Estos se utilizan internamente para operaciones de comparación entre individuos.

Las aptitudes ponderadas pueden compararse lexicográficamente utilizando los siguientes operadores: <, >, <= ,>=, ==, !=

## Creando la clase individual

El segundo uso común de la herramienta de "creator" en DEAP es definir los individuos que conforman la población del algoritmo genético. Los individuos en los algoritmos genéticos se representan mediante un cromosoma que puede manipularse mediante operadores genéticos.

En DEAP, la clase "Individual" se crea extendiendo una clase base que representa el cromosoma. Además, cada instancia individual en DEAP debe contener su función de aptitud como atributo.

Para cumplir estos dos requisitos, utilizamos el "creator" para crear la clase "creator.Individual", como se muestra en este ejemplo:



In [28]:
creator.create("Individual", list, fitness=creator.FitnessMax)

##### Cuestiones a tomar en cuenta

La anterior línea de código produce los siguientes dos efectos:
- La clase Individual creada extiende la clase "list" de Python.
- Esto significa que el cromosoma utilizado es del tipo lista.
- Cada instancia de esta clase Individual tendrá un atributo llamado fitness, de la clase FitnessMax

## Uso de la clase Toolbox

La clase "base.Toolbox" se utiliza como contenedor de funciones (u operadores) y permite crear nuevos operadores mediante alias y personalización de funciones existentes

In [ ]:
#Por ejemplo, supongamos que tenemos una función, sumOfTwo(), definida de la siguiente manera:
def sumOfTwo(a, b):
    return a + b 

#Usando la caja de herramientas, ahora podemos crear un nuevo operador, incrementByFive(),
# que personaliza la función sumOfTwo() de la siguiente manera:

from deap import base
toolbox= base.Toolbox()

toolbox.register("incrementByFive", sumOfTwo, b=5)


- El primer argumento que se pasa a la función register() es el nombre (o alias) deseado para el nuevo operador. 
- El segundo argumento es la función existente que se va a personalizar. 
- Cada argumento adicional (opcional) se pasa automáticamente a la función personalizada cada vez que se llama al nuevo operador.

Entonces, si llamo a la siguiente función:

In [30]:
 toolbox.incrementByFive(10)

15

Que es lo mismo que llamar a la función:

In [31]:
 sumOfTwo(10, 5)

15

Esto se debe a que el argumento b se ha fijado en un valor de 5 mediante la definición del operador incrementByFive.

## Creando Operadores Genéticos

En muchos casos, la clase Toolbox se utiliza para personalizar funciones existentes del módulo de herramientas. Este módulo contiene numerosas funciones útiles relacionadas con las operaciones genéticas de selección, cruce y mutación, así como utilidades de inicialización. Por ejemplo, el siguiente código define tres alias que se utilizarán posteriormente como operadores genéticos:

In [34]:
from deap import tools
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.02)

Los tres alias se definen en detalle a continuación:
- select se registra como alias de la función de herramientas existente, selTournament(), con el argumento tournsize establecido en 3. Esto crea un operador toolbox.select que realiza la selección de torneos con un tamaño de torneo de 3.

- mate se registra como alias de la función de herramientas existente, cxTwoPoint(). Esto genera un operador toolbox.mate que realiza un cruce de dos puntos.

- mutate se registra como alias de la función de herramientas existente, mutFlipBit, con el argumento indpb establecido en 0.02, lo que proporciona un operador toolbox.mutate que realiza la mutación de bit de inversión con una probabilidad de inversión de 0.02 para cada atributo.

### Algunas funciones de DEAP

Las funciones de selección se encuentran en el archivo selection.py. Algunas de ellas se listan a continuación:

- selRoulette() implementa la selección de la ruleta.
- selStochasticUniversalSampling() implementa el Muestreo Universal Estocástico (SUS).
- selTournament() implementa la selección del torneo.

Las funciones de cruce se encuentran en el archivo crossover.py:
 
- cxOnePoint() implementa el cruce de un solo punto.
- cxUniform() implementa el cruce uniforme.
- cxOrdered() implementa el cruce ordenado (OX1).
- cxPartialyMatched() implementa el cruce parcialmente coincidente (PMX).

Algunas de las funciones de mutación que se encuentran en el archivo mutation.py son:

- mutFlipBit() implementa la mutación de bit invertido.
- mutGaussian() implementa la mutación de distribución normal.


## Creando una población

El archivo "init.py" del módulo de herramientas contiene varias funciones útiles para crear e inicializar la población del algoritmo genético.

Una función especialmente útil es "initRepeat()", que acepta tres argumentos:

- El tipo de contenedor en el que se guardarán los objetos resultantes-
- La función utilizada para generar los objetos que se guardarán en el contenedor
- El número de objetos que se generarán.

Por ejemplo, la siguiente línea de código generará una lista de 30 números aleatorios entre 0 y 1.

In [37]:
import random
randomList = tools.initRepeat(list, random.random, 30)

¿Qué pasaría si quisiéramos llenar la lista con números enteros aleatorios que sean 0 o 1? Podríamos, por ejemplo, crear una función que utilice random.radint() para generar un único valor aleatorio de 0 o 1, y luego usarla como función generadora de initRepeat(), como se muestra en el siguiente fragmento de código:


In [40]:
def zeroOrOne():
 return random.randint(0, 1)

randomList = tools.initRepeat(list, zeroOrOne, 30)

## Cálculo de fitness

Como mencionamos anteriormente, si bien la clase Fitness define los pesos de aptitud que determinan su estrategia (como maximización o minimización), los valores de aptitud reales se obtienen de una función definida por separado. Esta función de cálculo de aptitud suele registrarse en el módulo de la caja de herramientas mediante el alias "evaluar", como se muestra en el siguiente fragmento de código:


In [42]:
def someFitnessCalculationFunction(individual):
    return _some_calculation_of_the_fitness

toolbox.register("evaluate",someFitnessCalculationFunction)

En este ejemplo, someFitnessCalculationFunction() calcula la aptitud física de cualquier individuo, y "evaluar" se registra como su alias.